In [1]:
#%display latex
import random

p = 2
m = 5
q = p^m

K = GF(p)
F.<z> = K.extension(m)

## Key generation

In [2]:
n = 30
assert (n % 2 == 0) and (n <= q-1) 
k = 10
X = random.sample([i for i in F if i != 0], n) #the support of RS code

Csec = codes.GeneralizedReedSolomonCode(X, k)
Qlist = []
for i in range(n//2):
    Qlist.append(
        block_diagonal_matrix(random_matrix(K, m, algorithm='unimodular'), random_matrix(K, m, algorithm='unimodular')) *
        block_diagonal_matrix(random_matrix(K, m-1, algorithm='unimodular'), random_matrix(K, m+1, algorithm='unimodular'))
    )
Q = block_diagonal_matrix(Qlist)

expMat = lambda G: block_matrix([[j.matrix().T for j in i] for i in G]) #returns subfield image of G

S = random_matrix(K, k*m, algorithm='unimodular')
Gpub = S * expMat(Csec.generator_matrix()) * Q
Cpub = codes.LinearCode(Gpub)
Cpub

[150, 50] linear code over GF(2)

## Step 1: Recovering the full support

Suppose we are given the partial support $x' = (x_1, x_3, x_5, \dots, x_{n-1})$ after applying CL–attack to code $D$ (see details in the paper). The goal of this step is to recover the remaing part of $x$, i.e. $x_2, x_4, \dots, x_n$.

In [3]:
Xodd = X[0::2] #odd coordinates of X

Let's try to recover $x_2$ using the procedure describded is Section 5.1 

In [4]:
coords = [*range(2*m)]
for i in range(2, n-1, 2):
    coords += [*range(i*m, i*m+(m-1))]
#coords

In [5]:
Godd2 = Gpub[:, coords]
D2 = codes.LinearCode(Godd2)
D2

[66, 50] linear code over GF(2)

In [6]:
GD2 = D2.parity_check_matrix()

In [7]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def fold(v, m=m):
    """Folds a vector of uknowns into matrix of form X_1 (see Section 5.1)"""
    m1 = matrix(m, m-1, v[:m^2-m])
    m2 = matrix(m, m+1, v[m^2-m:2*m^2])
    m3 = zero_matrix(m, m-1)
    m4 = matrix(m, m+1, v[2*m^2:3*m^2+m])
    return block_matrix([[m1,m2],[m3,m4]])

def foldM2(v, m=m):
    """ Folds a vector of uknowns into block diagonal matrix (X_1^T, X_2^T, X_3^T, ..., X^{n-1}^T) (see Section 5.1)"""
    sz = 3*m^2 + m
    M = []
    M.append(fold(v[:sz], m))
    for i in chunks(v[sz:], m*(m-1)):
        M.append(matrix(m, m-1, i))
    return block_diagonal_matrix(M)

sz = 3*m^2+m
v = random_vector(K, sz + m*(m-1)*2)
foldM2(v)

20 x 18 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

In [8]:
sz = 3*m^2+m + (n//2-1)*(m-1)*m
Isz = identity_matrix(K, sz)

for x2 in F:
    if x2 not in Xodd and (x2 != 0):
        RS_supp = [Xodd[0], x2] + Xodd[1:]
        RS_mat = expMat(codes.GeneralizedReedSolomonCode(RS_supp, k).generator_matrix())
        T = matrix([(GD2 * foldM2(i).T * RS_mat.T).list() for i in Isz])
        dm = T.left_kernel().dimension() #checking whether system 
                                        #G_{D_2} \cdot \diag\left( X_1^\tr, X_3^{\tr}, \dots, X_{n-1}^{\tr} \right) \cdot
                                        #\Exp_{\Gamma}\left( G_k(x_1, w, x_3, x_5, \dots, x_{n-1}) \right)^{\tr} = 0 has a solutions
        if dm > 0:
            print(x2 == X[1])
            break

True


So, $x_2$ is succesfully recovered. Let's find the rest by reducing the problem to the previous one.

In [9]:
Xeven = [x2]

def findX2(Gpub, Xodd):
    coords = [*range(2*m)]
    for i in range(2, n-1, 2):
        coords += [*range(i*m, i*m+(m-1))]
    coords
    Godd2 = Gpub[:, coords]
    GD2 = codes.LinearCode(Godd2).parity_check_matrix()
    sz = 3*m^2+m + (n//2-1)*(m-1)*m
    Isz = identity_matrix(K, sz)
    for x2 in F:
        if x2 not in Xodd and (x2 != 0):
            RS_supp = [Xodd[0], x2] + Xodd[1:]
            RS_mat = expMat(codes.GeneralizedReedSolomonCode(RS_supp, k).generator_matrix())
            T = matrix([(GD2 * foldM2(i).T * RS_mat.T).list() for i in Isz])
            dm = T.left_kernel().dimension() 
            if dm > 0:
                return x2
    return None

for i in range(1, n//2):
    Gpub_new = copy(Gpub)
    Xodd_new = copy(Xodd)
    Xodd_new[0], Xodd_new[i] = Xodd_new[i], Xodd_new[0]
    Gpub_new[:, 0:2*m], Gpub_new[:, 2*m*i:2*m*(i+1)] = Gpub_new[:, 2*m*i:2*m*(i+1)], Gpub_new[:, 0:2*m]
    Xeven.append(findX2(Gpub_new, Xodd_new))

In [10]:
X_recovered = [0]*n
X_recovered[0::2] = Xodd
X_recovered[1::2] = Xeven

X_recovered == X

True

Success! 

## Step 2: Recovering Q

In [11]:
Csec_rec = codes.GeneralizedReedSolomonCode(X_recovered, k)
RS_mat = expMat(Csec_rec.generator_matrix())
Hpub = Cpub.parity_check_matrix()

def foldM(v, m=m):
    sz = 3*m^2 + m
    M = []
    for i in chunks(v, sz):
        M.append(fold(i, m))
    return block_diagonal_matrix(M)

sz_b = 3*m^2+m
sz = n // 2 * sz_b
Isz = identity_matrix(K, sz)
X = matrix([(Hpub * foldM(i).T * (RS_mat.T)).list() for i in Isz])
X.left_kernel().dimension()

5

So, since number of solutions is $p^m = |\mathbb{F}_q|$ solution is found up to multiplication by some $\alpha \in \mathbb{F}_q$. Indeed, let's take a non-zero solution $Q_{restored}$

In [12]:
Qrestored = foldM(X.left_kernel().basis()[1])

In [13]:
alpha = F(Qrestored.solve_left(Q)[0, :m].list())
alpha

z^4 + z^3 + z^2

In [14]:
U = block_diagonal_matrix([alpha.matrix().T for i in range(n)])

U*Qrestored == Q

True